In [ ]:
# import Pkg
# using Pkg
# Pkg.add("FFMPEG_jll")
# Pkg.add("IterTools")
# Pkg.add("MAT")
# Pkg.add("OrderedCollections")
# Pkg.add("PlotlyJS")
# Pkg.add("Plots")
# Pkg.add("PyPlot")
# Pkg.build("FFMPEG_jll")
# Pkg.build("Plots")
# Pkg.update()
# Pkg.precompile()
# Pkg.status()
# Pkg.add("DSP")
# Sys.BINDIR

In [ ]:
using Pkg
using DSP
using IterTools
using LinearAlgebra
using MAT
using OrderedCollections
using Plots
using Statistics
gr()
plotlyjs()

In [ ]:
function T(x, i, j)
    if abs(j) > 100
        error("Shift value must be within the range [-100, 100].")
    end
    x_padded = vcat(zeros(100), x, zeros(100)) # Padding 100 zeros before and after x
    mid = div(length(x), 2)
    if i == 0 # Take all of x
        y = x_padded[101-j:end-100-j]
    elseif i == 1 # Take the first half of x
        y = x_padded[101-j:100+mid-j]
    elseif i == 2 # Take the second half of x
        y = x_padded[101+mid-j:end-100-j]
    else
        error("j must be either 0, 1 or 2.")
    end
    return y
end

## PA

In [ ]:
# PA: MP
function PA(x, c)
    X = zeros(ComplexF64, length(x), 15)
    p = 1
    for (k, i) in product(0:4, 0:2)
        X[:, p] = abs.(T(x,0,i)).^(2k) .* T(x,0,i)
        p += 1
    end
    y = X * c
    return y
end

In [ ]:
# PA: GMP
function PA(x, c)
    X = zeros(ComplexF64, length(x), 136)
    p = 1
    for (ka, la) in product(0:5, 0:5)
        X[:, p] = T(x,0,la) .* abs.(T(x,0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:5, 0:3, 1:2)
        X[:, p] = T(x,0,lb) .* abs.(T(x,0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:5, 0:5, 1:2)
        X[:, p] = T(x,0,lc) .* abs.(T(x,0,lc-mc)).^(2kc)
        p += 1
    end
    y = X * c
    return y
end

In [ ]:
# PA: 2nd DDR
function PA(x, c)
    X = zeros(ComplexF64, length(x), 219)
    p = 1 # Position of the specific column to be filled
    for (k, i) in product(0:24, 0:2)
        X[:, p] = T(x,0,i) .* abs.(T(x,0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:24, 1:2)
        X[:, p] = conj(T(x,0,i)) .* T(x,0,0).^2 .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:24, 1:2)
        X[:, p] = abs.(T(x,0,i)).^2 .* T(x,0,0) .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:24, 1:2)
        X[:, p] = T(x,0,i).^2 .* conj(T(x,0,0)) .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    y = X * c
    return y
end

In [ ]:
# PA: VTD 2nd DDR
function PA(x, c)
    TH = [0, 0.74, 1]
    xS = repeat(x, 1, 2)
    for s in 1:length(TH)-1
        xS[abs.(xS[:, s]) .<= TH[s], s] .= 0
        xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s] .= (abs.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]) .- TH[s]) .* exp.(1im * angle.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]))
        xS[abs.(xS[:, s]) .> TH[s+1], s] .= (TH[s+1] - TH[s]) .* exp.(1im * angle.(xS[abs.(xS[:, s]) .> TH[s+1], s]))
    end
    X = zeros(ComplexF64, length(x), 312)
    p = 1 # Position of the specific column to be filled
    for (k, i) in product(0:10, 0:2)
        X[:, p] = T(xS[:, 1],0,i) .* abs.(T(xS[:, 1],0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:10, 1:2)
        X[:, p] = conj(T(xS[:, 1],0,i)) .* T(xS[:, 1],0,0).^2 .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:10, 1:2)
        X[:, p] = abs.(T(xS[:, 1],0,i)).^2 .* T(xS[:, 1],0,0) .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:10, 1:2)
        X[:, p] = T(xS[:, 1],0,i).^2 .* conj(T(xS[:, 1],0,0)) .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(0:24, 0:2)
        X[:, p] = T(xS[:, 2],0,i) .* abs.(T(xS[:, 2],0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:24, 1:2)
        X[:, p] = conj(T(xS[:, 2],0,i)) .* T(xS[:, 2],0,0).^2 .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:24, 1:2)
        X[:, p] = abs.(T(xS[:, 2],0,i)).^2 .* T(xS[:, 2],0,0) .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:24, 1:2)
        X[:, p] = T(xS[:, 2],0,i).^2 .* conj(T(xS[:, 2],0,0)) .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    y = X * c
    return y
end

In [ ]:
# PA: VTD GMP
function PA(x, c)
    TH = [0, 0.74, 1]
    xS = repeat(x, 1, 2)
    for s in 1:length(TH)-1
        xS[abs.(xS[:, s]) .<= TH[s], s] .= 0
        xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s] .= (abs.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]) .- TH[s]) .* exp.(1im * angle.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]))
        xS[abs.(xS[:, s]) .> TH[s+1], s] .= (TH[s+1] - TH[s]) .* exp.(1im * angle.(xS[abs.(xS[:, s]) .> TH[s+1], s]))
    end
    X = zeros(ComplexF64, length(x), 305)
    p = 1
    for (ka, la) in product(0:13, 0:2)
        X[:, p] = T(xS[:, 1],0,la) .* abs.(T(xS[:, 1],0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:11, 0:1, 1:1)
        X[:, p] = T(xS[:, 1],0,lb) .* abs.(T(xS[:, 1],0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:14, 0:2, 1:2)
        X[:, p] = T(xS[:, 1],0,lc) .* abs.(T(xS[:, 1],0,lc-mc)).^(2kc)
        p += 1
    end
    for (ka, la) in product(0:12, 0:2)
        X[:, p] = T(xS[:, 2],0,la) .* abs.(T(xS[:, 2],0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:14, 0:0, 1:2)
        X[:, p] = T(xS[:, 2],0,lb) .* abs.(T(xS[:, 2],0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:15, 0:2, 1:2)
        X[:, p] = T(xS[:, 2],0,lc) .* abs.(T(xS[:, 2],0,lc-mc)).^(2kc)
        p += 1
    end
    y = X * c
    return y
end

## DPD MP

In [ ]:
DPD_model_name = "MP"
function makeX(x)
    X = zeros(ComplexF64, length(x), 12)
    p = 1
    for (k, i) in product(0:3, 0:2)
        X[:, p] = abs.(T(x,0,i)).^(2k) .* T(x,0,i)
        p += 1
    end
    return X
end

## DPD GMP

In [ ]:
DPD_model_name = "GMP"
function makeX(x)
    X = zeros(ComplexF64, length(x), 641) # 641
    p = 1
    for (ka, la) in product(0:10, 0:10)
        X[:, p] = T(x,0,la) .* abs.(T(x,0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:10, 0:5, 1:5)
        X[:, p] = T(x,0,lb) .* abs.(T(x,0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:10, 0:10, 1:2)
        X[:, p] = T(x,0,lc) .* abs.(T(x,0,lc-mc)).^(2kc)
        p += 1
    end
    return X
end

## DPD 2nd DDR

In [ ]:
DPD_model_name = "2nd DDR"
function makeX(x)
    X = zeros(ComplexF64, length(x), 421)
    p = 1 # Position of the specific column to be filled
    for (k, i) in product(0:10, 0:10)
        X[:, p] = T(x,0,i) .* abs.(T(x,0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:10, 1:10)
        X[:, p] = conj(T(x,0,i)) .* T(x,0,0).^2 .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:10, 1:10)
        X[:, p] = abs.(T(x,0,i)).^2 .* T(x,0,0) .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:10, 1:10)
        X[:, p] = T(x,0,i).^2 .* conj(T(x,0,0)) .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    return X
end

In [ ]:
DPD_model_name = "VTD 2nd DDR"
function makeX(x)
    TH = [0, 0.54, 1]
    xS = repeat(x, 1, 2)
    for s in 1:length(TH)-1
        xS[abs.(xS[:, s]) .<= TH[s], s] .= 0
        xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s] .= (abs.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]) .- TH[s]) .* exp.(1im * angle.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]))
        xS[abs.(xS[:, s]) .> TH[s+1], s] .= (TH[s+1] - TH[s]) .* exp.(1im * angle.(xS[abs.(xS[:, s]) .> TH[s+1], s]))
    end
    X = zeros(ComplexF64, length(x), 276)
    p = 1 # Position of the specific column to be filled
    for (k, i) in product(0:5, 0:2)
        X[:, p] = T(xS[:, 1],0,i) .* abs.(T(xS[:, 1],0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:5, 1:2)
        X[:, p] = conj(T(xS[:, 1],0,i)) .* T(xS[:, 1],0,0).^2 .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:5, 1:2)
        X[:, p] = abs.(T(xS[:, 1],0,i)).^2 .* T(xS[:, 1],0,0) .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:5, 1:2)
        X[:, p] = T(xS[:, 1],0,i).^2 .* conj(T(xS[:, 1],0,0)) .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(0:25, 0:2)
        X[:, p] = T(xS[:, 2],0,i) .* abs.(T(xS[:, 2],0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:25, 1:2)
        X[:, p] = conj(T(xS[:, 2],0,i)) .* T(xS[:, 2],0,0).^2 .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:25, 1:2)
        X[:, p] = abs.(T(xS[:, 2],0,i)).^2 .* T(xS[:, 2],0,0) .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:25, 1:2)
        X[:, p] = T(xS[:, 2],0,i).^2 .* conj(T(xS[:, 2],0,0)) .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    return X
end

In [ ]:
DPD_model_name = "VTD GMP"
function makeX(x)
    TH = [0, 0.54, 1]
    xS = repeat(x, 1, 2)
    for s in 1:length(TH)-1
        xS[abs.(xS[:, s]) .<= TH[s], s] .= 0
        xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s] .= (abs.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]) .- TH[s]) .* exp.(1im * angle.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]))
        xS[abs.(xS[:, s]) .> TH[s+1], s] .= (TH[s+1] - TH[s]) .* exp.(1im * angle.(xS[abs.(xS[:, s]) .> TH[s+1], s]))
    end
    X = zeros(ComplexF64, length(x), 319)
    p = 1
    for (ka, la) in product(0:10, 0:2)
        X[:, p] = T(xS[:, 1],0,la) .* abs.(T(xS[:, 1],0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:13, 0:1, 1:1)
        X[:, p] = T(xS[:, 1],0,lb) .* abs.(T(xS[:, 1],0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:11, 0:2, 1:2)
        X[:, p] = T(xS[:, 1],0,lc) .* abs.(T(xS[:, 1],0,lc-mc)).^(2kc)
        p += 1
    end
    for (ka, la) in product(0:15, 0:2)
        X[:, p] = T(xS[:, 2],0,la) .* abs.(T(xS[:, 2],0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:19, 0:0, 1:2)
        X[:, p] = T(xS[:, 2],0,lb) .* abs.(T(xS[:, 2],0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:18, 0:2, 1:2)
        X[:, p] = T(xS[:, 2],0,lc) .* abs.(T(xS[:, 2],0,lc-mc)).^(2kc)
        p += 1
    end
    return X
end

## MDDR

In [ ]:
DPD_model_name = "MDDR"
function makeX(x)
    X = zeros(ComplexF64, length(x), 641) # 641
    p = 1
    for (ka, qa) in product(0:10, 0:10)
        X[:, p] = abs.(T(x,0,qa)).^(2ka) .* T(x,0,qa)
        p += 1
    end
    for (kb, qb, mb) in product(1:10, 0:5, 1:5)
        X[:, p] = abs.(T(x,0,qb+mb)).^(2*(kb-1)) .* (T(x,0,0)).^2 .* conj(T(x,0,qb))
        p += 1
    end
    for (kc, qc, mc) in product(1:10, 0:10, 1:2)
        X[:, p] = abs.(T(x,0,qc-mc)).^(2*(kc-1)) .* (T(x,0,0)).^2 .* conj(T(x,0,qc))
        p += 1
    end
    return X
end

## CRV

In [ ]:
DPD_model_name = "CRV"
function makeX(x)
    X = zeros(ComplexF64, length(x), 616)
    p = 1
    for i in 1:2:11
        X[:, p] = T(x,0,0) .^((i+1)/2) .* conj(T(x,0,0)).^((i-1)/2)
        p += 1
    end
    for (i, m, k) in product(1:2:9, 1:10, 2:2:10)
        X[:, p] = T(x,0,0) .^((i+1)/2) .* conj(T(x,0,0)).^((i-1)/2) .* T(x,0,m) .^(k/2) .* conj(T(x,0,m)).^(k/2)
        p += 1
    end
    for (i, m, k) in product(0:2:10, 1:10, 1:2:11)
        X[:, p] = T(x,0,0) .^(i/2) .* conj(T(x,0,0)).^(i/2) .* T(x,0,m) .^((k+1)/2) .* conj(T(x,0,m)).^((k-1)/2)
        p += 1
    end
    return X
end

## MSA

In [ ]:
DPD_model_name = "MSA"
function HK(Φ, ψ)
    X = zeros(Complex{Float64}, length(Φ), 40)
    intervals = [(-Inf, 0.1)]
    append!(intervals, [(i/10, (i+1)/10) for i in 1:8])
    push!(intervals, (0.9, Inf))
    col_idx = 1
    for interval in intervals
        indices = findall(x -> x > interval[1] && x <= interval[2], abs.(ψ))
        for idx in indices
            angle_value = angle.(Φ[idx])
            X[idx[1], col_idx] = Φ[idx]
            X[idx[1], col_idx + 1] = exp.(im * angle_value)
            X[idx[1], col_idx + 2] = Φ[idx] .* abs.(ψ[idx])
            X[idx[1], col_idx + 3] = exp.(im * angle_value) .* abs.(ψ[idx])
        end
        col_idx += 4
    end
    return X
end
function makeX(x)
    X = zeros(ComplexF64, length(x), 451)
    p = 1
    for m in 0:10
        X[:, p] = T(x,0,m)
        p += 1
    end
    for m in 0:10
        X[:, p:p+39] = HK(T(x,0,m), T(x,0,0))
        p += 40
    end
    return X
end

## EMSA

In [ ]:
DPD_model_name = "EMSA"
function HK1(Φ, ψ)
    X = zeros(Complex{Float64}, length(Φ), 20)
    intervals = [(-Inf, 0.1)]
    append!(intervals, [(i/10, (i+1)/10) for i in 1:8])
    push!(intervals, (0.9, Inf))
    col_idx = 1
    for interval in intervals
        indices = findall(x -> x >= interval[1] && x < interval[2], ψ)
        for idx in indices
            angle_value = angle.(Φ[idx])
            X[idx[1], col_idx] = Φ[idx]
            X[idx[1], col_idx + 1] = exp.(im * angle_value)
        end
        col_idx += 2
    end
    return X
end
function HK2(Φ, ψ)
    X = zeros(Complex{Float64}, length(Φ), 20)
    intervals = [(-Inf, -0.09)]
    append!(intervals, [((i/10)^2 - 0.1, ((i+1)/10)^2 - 0.1) for i in 1:8])
    push!(intervals, (0.71, Inf))
    col_idx = 1
    for interval in intervals
        indices = findall(x -> x >= interval[1] && x < interval[2], ψ)
        for idx in indices
            angle_value = angle.(Φ[idx])
            X[idx[1], col_idx] = Φ[idx]
            X[idx[1], col_idx + 1] = exp.(im * angle_value)
        end
        col_idx += 2
    end
    return X
end
function makeX(x)
    X = zeros(ComplexF64, length(x), 1520)
    p = 1
    for m in 1:10
        X[:, p:p+19] = HK1(T(x,0,0), abs.(T(x,0,m)))
        p += 20
    end
    for m in 1:10
        X[:, p:p+19] = HK1(T(x,0,m), abs.(T(x,0,0)))
        p += 20
    end
    for m in 1:10
        X[:, p:p+19] = HK2(T(x,0,0), real.(T(x,0,0) .* conj(T(x,0,m))))
        p += 20
    end
    for m in 1:10
        X[:, p:p+19] = HK2(T(x,0,m), real.(T(x,0,0) .* conj(T(x,0,m))))
        p += 20
    end
    for (l, m) in product(0:5, 0:5)
        X[:, p:p+19] = HK1(T(x,0,m+l), abs.(T(x,0,m)))
        p += 20
    end
    return X
end

## DVR

In [ ]:
DPD_model_name = "DVR"
function makeX(x)
    X = zeros(ComplexF64, length(x), 531)
    p = 1
    for m in 0:10
        X[:, p] = T(x,0,m)
        p += 1
    end
    for (k, m) in product(1:10, 0:10)
        X[:, p] = abs.(abs.(T(x,0,m)) .- k/10) .* exp.(im * angle.(T(x,0,m)))
        p += 1
    end
    for (k, m) in product(1:10, 0:10)
        X[:, p] = abs.(abs.(T(x,0,m)) .- k/10) .* exp.(im * angle.(T(x,0,m))) .* abs.(T(x,0,0))
        p += 1
    end
    for (k, m) in product(1:10, 1:10)
        X[:, p] = abs.(abs.(T(x,0,m)) .- k/10) .* T(x,0,0)
        p += 1
    end
    for (k, m) in product(1:10, 1:10)
        X[:, p] = abs.(abs.(T(x,0,m)) .- k/10) .* T(x,0,m)
        p += 1
    end
    for (k, m) in product(1:10, 1:10)
        X[:, p] = abs.(abs.(T(x,0,0)) .- k/10) .* T(x,0,m)
        p += 1
    end
    return X
end

## Hybrid

In [ ]:
DPD_model_name = "Hybrid"
function makeX(x)
    X = zeros(ComplexF64, length(x), 2319)
    p = 1
    for (ka, la) in product(0:10, 0:10)
        X[:, p] = T(x,0,la) .* abs.(T(x,0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:10, 0:5, 1:5)
        X[:, p] = T(x,0,lb) .* abs.(T(x,0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:10, 0:10, 1:2)
        X[:, p] = T(x,0,lc) .* abs.(T(x,0,lc-mc)).^(2kc)
        p += 1
    end
    for (k, i) in product(0:10, 0:10)
        X[:, p] = T(x,0,i) .* abs.(T(x,0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:10, 1:10)
        X[:, p] = conj(T(x,0,i)) .* T(x,0,0).^2 .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:10, 1:10)
        X[:, p] = abs.(T(x,0,i)).^2 .* T(x,0,0) .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:10, 1:10)
        X[:, p] = T(x,0,i).^2 .* conj(T(x,0,0)) .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    for (ka, qa) in product(0:10, 0:10)
        X[:, p] = abs.(T(x,0,qa)).^(2ka) .* T(x,0,qa)
        p += 1
    end
    for (kb, qb, mb) in product(1:10, 0:5, 1:5)
        X[:, p] = abs.(T(x,0,qb+mb)).^(2*(kb-1)) .* (T(x,0,0)).^2 .* conj(T(x,0,qb))
        p += 1
    end
    for (kc, qc, mc) in product(1:10, 0:10, 1:2)
        X[:, p] = abs.(T(x,0,qc-mc)).^(2*(kc-1)) .* (T(x,0,0)).^2 .* conj(T(x,0,qc))
        p += 1
    end
    for i in 1:2:11
        X[:, p] = T(x,0,0) .^((i+1)/2) .* conj(T(x,0,0)).^((i-1)/2)
        p += 1
    end
    for (i, m, k) in product(1:2:9, 1:10, 2:2:10)
        X[:, p] = T(x,0,0) .^((i+1)/2) .* conj(T(x,0,0)).^((i-1)/2) .* T(x,0,m) .^(k/2) .* conj(T(x,0,m)).^(k/2)
        p += 1
    end
    for (i, m, k) in product(0:2:10, 1:10, 1:2:11)
        X[:, p] = T(x,0,0) .^(i/2) .* conj(T(x,0,0)).^(i/2) .* T(x,0,m) .^((k+1)/2) .* conj(T(x,0,m)).^((k-1)/2)
        p += 1
    end
    num_rows, num_cols = size(X)
    columns_to_keep = Bool[]
    for i in 1:num_cols
        is_duplicate = false
        for j in 1:i-1
            sample_indices = rand(1:num_rows, 10)
            difference = abs.(X[sample_indices, i] - X[sample_indices, j])
            if all(difference .< 1e-100)
                is_duplicate = true
                break
            end
        end
        push!(columns_to_keep, !is_duplicate)
    end
    return X[:, columns_to_keep]
end

## GMP_MDDR

In [ ]:
DPD_model_name = "GMP_MDDR"
function makeX(x)
    X = zeros(ComplexF64, length(x), 1282)
    p = 1
    for (ka, la) in product(0:10, 0:10)
        X[:, p] = T(x,0,la) .* abs.(T(x,0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:10, 0:5, 1:5)
        X[:, p] = T(x,0,lb) .* abs.(T(x,0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:10, 0:10, 1:2)
        X[:, p] = T(x,0,lc) .* abs.(T(x,0,lc-mc)).^(2kc)
        p += 1
    end
    for (ka, qa) in product(0:10, 0:10)
        X[:, p] = abs.(T(x,0,qa)).^(2ka) .* T(x,0,qa)
        p += 1
    end
    for (kb, qb, mb) in product(1:10, 0:5, 1:5)
        X[:, p] = abs.(T(x,0,qb+mb)).^(2*(kb-1)) .* (T(x,0,0)).^2 .* conj(T(x,0,qb))
        p += 1
    end
    for (kc, qc, mc) in product(1:10, 0:10, 1:2)
        X[:, p] = abs.(T(x,0,qc-mc)).^(2*(kc-1)) .* (T(x,0,0)).^2 .* conj(T(x,0,qc))
        p += 1
    end
    num_rows, num_cols = size(X)
    columns_to_keep = Bool[]
    for i in 1:num_cols
        is_duplicate = false
        for j in 1:i-1
            sample_indices = rand(1:num_rows, 10)
            difference = abs.(X[sample_indices, i] - X[sample_indices, j])
            if all(difference .< 1e-100)
                is_duplicate = true
                break
            end
        end
        push!(columns_to_keep, !is_duplicate)
    end
    return X[:, columns_to_keep]
end

## Initialize

In [ ]:
data_PA = matread("300M_PA.mat")
data_c = matread("c.mat")
c = data_c["c"]
x = ComplexF64.(data_PA["x_in"])
x = x / maximum(abs.(x))
yp = PA(x, c)
yp = yp / maximum(abs.(yp))
N = length(x)
NMSE = 10 * log10(sum(abs2, yp - T(x,0,0)) / sum(abs2, T(x,0,0)))
println("The NMSE of PA data is ", NMSE, " dB.")
z = copy(x)
y = PA(z, c)
ynor = y / maximum(abs.(y))
D = makeX(x)
w = zeros(ComplexF64, size(D, 2))
μ = 0.01
l = 0.01
S = collect(1:size(D, 2))
NMSE = 0

## Iterate once at μ = 1

In [ ]:
z_former, w_former, y_former, ynor_former = copy(z), copy(w), copy(y), copy(ynor)
μ = 1
NMSE_μ = 0
z_μ, w_μ, y_μ, ynor_μ = copy(z_former), copy(w_former), copy(y_former), copy(ynor_former)
z_temp, w_temp, y_temp, ynor_temp = copy(z_former), copy(w_former), copy(y_former), copy(ynor_former)
X = makeX(ynor_temp)
X = X[:, S]
z_ = X * w_temp
e = z_temp - z_
w_temp = w_μ + μ * pinv(X' * X + 1 * I(size(X, 2))) * X' * e
z_temp = D * w_temp
w_temp = w_temp / maximum(abs.(z_temp))
z_temp = D * w_temp
y_temp = PA(z_temp, c)
ynor_temp = y_temp / maximum(abs.(y_temp))
NMSE_temp = 10 * log10(sum(abs2, ynor_temp - x) / sum(abs2, x))
if NMSE_temp < NMSE_μ
    z_μ, w_μ, y_μ, ynor_μ, NMSE_μ = copy(z_temp), copy(w_temp), copy(y_temp), copy(ynor_temp), copy(NMSE_temp)
end
if NMSE_μ < NMSE
    z, w, y, ynor, NMSE = copy(z_μ), copy(w_μ), copy(y_μ), copy(ynor_μ), copy(NMSE_μ)
    println("$μ   $NMSE")
end

## Read data

In [ ]:
# Read the DPD model corresponding to w matrix
# data_w = matread("w_GMP.mat")
# w = data_w["w_GMP"];

## Filter the most influential coefficients

In [ ]:
S = Int64[]
# Model inverse structure with indirect learning
# X = makeX(ynor)
# y_ = copy(x)
# Model reference structure
X = makeX(x)
y_ = copy(ynor - x)

Z = copy(X)
r = copy(y_)
for _ = 1:100
    for i = 1:size(X, 2)
        if !(i in S)
            Z[:, i] = Z[:, i] / norm(Z[:, i])
        end
    end
    indices = sortperm(abs.(vec(Z' * r)), rev=true)
    i = 0
    for index in indices
        if !(index in S)
            i = index
            break
        end
    end
    push!(S, i)
    p = Z[:, i]' * Z
    Z = Z - kron(p, Z[:, i])
    r = y_ - X[:, S] * pinv((X[:, S]' * X[:, S]) + 1 * I(size(X[:, S], 2))) * X[:, S]' * y_
end
z = copy(x)
y = PA(z, c)
ynor = y / maximum(abs.(y))
D = D[:, S]
w = zeros(ComplexF64, size(S))
NMSE = 0
S

## Train w matrix (Deactivated)

In [ ]:
# Fix μ and then iterate
z_former, w_former, y_former, ynor_former = copy(z), copy(w), copy(y), copy(ynor)
for μ = 0.01:0.01:1
    NMSE_history = Float64[]
    NMSE_μ = 0
    z_μ, w_μ, y_μ, ynor_μ = copy(z_former), copy(w_former), copy(y_former), copy(ynor_former)
    z_temp, w_temp, y_temp, ynor_temp = copy(z_former), copy(w_former), copy(y_former), copy(ynor_former)
    for _ = 1:1000
        X = makeX(ynor_temp)
        X = X[:, S]
        z_ = X * w_temp
        e = z_temp - z_
        w_temp = w_temp + μ * pinv(X' * X + 1 * I(size(X, 2))) * X' * e
        z_temp = D * w_temp
        w_temp = w_temp / maximum(abs.(z_temp))
        z_temp = D * w_temp
        y_temp = PA(z_temp, c)
        ynor_temp = y_temp / maximum(abs.(y_temp))
        NMSE_temp = 10 * log10(sum(abs2, ynor_temp - x) / sum(abs2, x))
        if NMSE_temp < NMSE_μ
            z_μ, w_μ, y_μ, ynor_μ, NMSE_μ = copy(z_temp), copy(w_temp), copy(y_temp), copy(ynor_temp), copy(NMSE_temp)
        end
        push!(NMSE_history, NMSE_temp)
        if length(NMSE_history) > 5 && NMSE_history[1] == minimum(NMSE_history)
            break
        end        
        if length(NMSE_history) > 5
            popfirst!(NMSE_history)
        end
    end
    if NMSE_μ < NMSE
        z, w, y, ynor, NMSE = copy(z_μ), copy(w_μ), copy(y_μ), copy(ynor_μ), copy(NMSE_μ)
        println("$μ   $NMSE")
    end
end

In [ ]:
# Fine Tuning: Fix l and then iterate
# z_former, w_former, y_former, ynor_former = copy(z), copy(w), copy(y), copy(ynor)
# for l = 0.01:0.01:1
#     NMSE_history = Float64[]
#     NMSE_l = 0
#     z_l, w_l, y_l, ynor_l = copy(z_former), copy(w_former), copy(y_former), copy(ynor_former)
#     z_temp, w_temp, y_temp, ynor_temp = copy(z_former), copy(w_former), copy(y_former), copy(ynor_former)
#     for _ = 1:1000
#         r = ynor_temp - x
#         a = pinv(D' * D + 1 * I(size(D, 2))) * D' * T(r,0,0)
#         w_temp = w_temp - l * a
#         z_temp = D * w_temp
#         w_temp = w_temp / maximum(abs.(z_temp))
#         z_temp = D * w_temp
#         y_temp = PA(z_temp ,c)
#         ynor_temp = y_temp / maximum(abs.(y_temp))
#         NMSE_temp = 10 * log10(sum(abs2, ynor_temp - x) / sum(abs2, x))
#         if NMSE_temp < NMSE_l
#             z_l, w_l, y_l, ynor_l, NMSE_l = copy(z_temp), copy(w_temp), copy(y_temp), copy(ynor_temp), copy(NMSE_temp)
#         end
#         push!(NMSE_history, NMSE_temp)
#         if length(NMSE_history) > 5 && NMSE_history[1] == minimum(NMSE_history)
#             break
#         end
#         if length(NMSE_history) > 5
#             popfirst!(NMSE_history)
#         end
#     end
#     if NMSE_l < NMSE
#         z, w, y, ynor, NMSE = copy(z_l), copy(w_l), copy(y_l), copy(ynor_l), copy(NMSE_l)
#         println("$l   $NMSE")
#     end
# end

## Train w matrix

In [ ]:
# Coarse Extraction
NMSE_history = Float64[]
for iteration = 1:1000
    z_temp, w_temp, y_temp, ynor_temp = copy(z), copy(w), copy(y), copy(ynor)
    X = makeX(ynor_temp)
    X = X[:, S]
    w_temp = pinv(X' * X + 1 * I(size(X, 2))) * X' * z_temp
    z_temp = D * w_temp
    w_temp = w_temp / maximum(abs.(z_temp))
    z_temp = D * w_temp
    y_temp = PA(z_temp, c)
    ynor_temp = y_temp / maximum(abs.(y_temp))
    NMSE_temp = 10 * log10(sum(abs2, ynor_temp - x) / sum(abs2, x))
    if NMSE_temp < NMSE
        z, w, y, ynor, NMSE = copy(z_temp), copy(w_temp), copy(y_temp), copy(ynor_temp), copy(NMSE_temp)
        println("$iteration   $NMSE")
    end
    push!(NMSE_history, NMSE_temp)
    if length(NMSE_history) > 5 && NMSE_history[1] == minimum(NMSE_history)
        break
    end
    if length(NMSE_history) > 5
        popfirst!(NMSE_history)
    end
end

In [ ]:
# Each iteration takes the optimal μ
for iteration = 1:1000
    μ_ = 0
    foundBetterμ = false
    z_former, w_former, y_former, ynor_former = copy(z), copy(w), copy(y), copy(ynor)
    for μ = 0.01:0.01:1
        z_μ, w_μ, y_μ, ynor_μ = copy(z_former), copy(w_former), copy(y_former), copy(ynor_former)
        X = makeX(ynor_μ)
        X = X[:, S]
        z_ = X * w_μ
        e = z_μ - z_
        w_μ = w_μ + μ * pinv(X' * X + 1 * I(size(X, 2))) * X' * e
        z_μ = D * w_μ
        w_μ = w_μ / maximum(abs.(z_μ))
        z_μ = D * w_μ
        y_μ = PA(z_μ, c)
        ynor_μ = y_μ / maximum(abs.(y_μ))
        NMSE_μ = 10 * log10(sum(abs2, ynor_μ - x) / sum(abs2, x))
        if NMSE_μ < NMSE
            z, w, y, ynor, NMSE, μ_ = copy(z_μ), copy(w_μ), copy(y_μ), copy(ynor_μ), copy(NMSE_μ), copy(μ)
            foundBetterμ = true
        end
    end
    if !foundBetterμ
        break
    end
    println("$iteration   $μ_   $NMSE")
end

In [ ]:
# Fine Tuning: Each iteration takes the optimal l
for iteration = 1:1000
    l_ = 0
    foundBetterl = false
    r = ynor - x
    a = pinv(D' * D + 1 * I(size(D, 2))) * D' * T(r,0,0)
    z_former, w_former, y_former, ynor_former = copy(z), copy(w), copy(y), copy(ynor)
    for l = 0.01:0.01:1
        z_l, w_l, y_l, ynor_l = copy(z_former), copy(w_former), copy(y_former), copy(ynor_former)
        w_l = w_l - l * a
        z_l = D * w_l
        w_l = w_l / maximum(abs.(z_l))
        z_l = D * w_l
        y_l = PA(z_l ,c)
        ynor_l = y_l / maximum(abs.(y_l))
        NMSE_l = 10 * log10(sum(abs2, ynor_l - x) / sum(abs2, x))
        if NMSE_l < NMSE
            z, w, y, ynor, NMSE, l_ = copy(z_l), copy(w_l), copy(y_l), copy(ynor_l), copy(NMSE_l), copy(l)
            foundBetterl = true
        end
    end
    if !foundBetterl
        break
    end
    println("$iteration   $l_   $NMSE")
end

In [ ]:
yn = hcat(yn, y);

## Plot

In [ ]:
# Plot and Print
PA_model_name = "GMP"

# Figure 1
p1 = scatter(abs.(T(x,0,0)), abs.(T(z,0,0)), markersize=1, color=:blue, markerstrokecolor=false, legend=false)
plot!(aspect_ratio=:equal, xlims=(0, 1), ylims=(0, 1))
title!("DPD: $(DPD_model_name)")
xlabel!("Normalized Input Magnitude")
ylabel!("z Magnitude")

# Figure 2
p2 = scatter(abs.(T(z,0,0)), abs.(T(y,0,0)), markersize=1, color=:blue, markerstrokecolor=false, legend=false)
plot!(aspect_ratio=:equal, xlims=(0, 1), ylims=(0, 1))
title!("PA: $(PA_model_name)")
xlabel!("z Magnitude")
ylabel!("Output Magnitude")

# Figure 3
p3 = scatter(abs.(T(x,0,0)), abs.(T(yp,0,0)), markersize=1, color=:blue, markerstrokecolor=false, legend=false)
scatter!(abs.(T(x,0,0)), abs.(T(ynor,0,0)), markersize=1, color=:green, markerstrokecolor=false, legend=false)
plot!(aspect_ratio=:equal, xlims=(0, 1), ylims=(0, 1))
title!("DPD: $(DPD_model_name)     PA: $(PA_model_name)")
xlabel!("Normalized Input Magnitude")
ylabel!("Normalized Output Magnitude")

# Figure 4
p4 = scatter(abs.(T(x,0,0)), rad2deg.(angle.(T(yp,0,0)./T(x,0,0))), markersize=1, color=:blue, markerstrokecolor=false, legend=false)
scatter!(abs.(T(x,0,0)), rad2deg.(angle.(T(ynor,0,0)./T(x,0,0))), markersize=1, color=:green, markerstrokecolor=false, legend=false)
plot!(ylims=(-180, 180), yticks=-180:60:180)
title!("DPD: $(DPD_model_name)     PA: $(PA_model_name)")
xlabel!("Normalized Input Magnitude")
ylabel!("Phase Difference (Degrees)")

# Display plots
display(p1)
display(p2)
display(p3)
display(p4)

# Print specific information
println("The minimum NMSE in this DPD model is ", NMSE, " dB.")

In [ ]:
coefficients = 10:10:100
GMP = [-38.3471, -40.3134, -42.6905, -42.4404, -44.3829, -44.6661, -45.2438, -45.6323, -45.4759, -45.3121]
second_DDR = [-31.6134, -33.3493, -37.9592, -38.8205, -39.2914, -39.9636, -39.7814, -39.7474, -39.4601, -39.1841]
MDDR = [-37.4724, -39.6167, -40.9633, -43.2278, -43.2585, -43.7902, -43.5494, -43.9594, -43.5406, -43.0498]
CRV = [-31.6134, -35.9507, -37.1913, -37.3986, -39.1248, -40.0129, -40.0296, -41.7389, -41.8669, -39.6275]
DVR = [-31.0672, -31.0628, -30.5571, -32.8093, -33.5114, -32.4486, -33.4949, -35.3198, -34.5923, -34.4975]
GMP_MDDR = [-39.1521, -39.4304, -41.8464, -43.6218, -42.9530, -41.9921, -43.4581, -44.0396, -44.0953, -44.4060]
hybrid = [-39.1085, -39.6270, -42.5610, -44.2799, -45.0888, -43.6370, -44.2092, -44.2604, -43.0847, -43.7773]
p_NMSE = plot()
plot!(p_NMSE, coefficients, GMP, label="GMP", color=:red, linewidth=2, marker=:utriangle, markersize=3, markercolor=:red, markerstrokecolor=:red)
plot!(p_NMSE, coefficients, second_DDR, label="2nd DDR", color=:blue, linewidth=2, marker=:utriangle, markersize=3, markercolor=:blue, markerstrokecolor=:blue)
plot!(p_NMSE, coefficients, MDDR, label="MDDR", color=:green, linewidth=2, marker=:utriangle, markersize=3, markercolor=:green, markerstrokecolor=:green)
plot!(p_NMSE, coefficients, CRV, label="CRV", color=:orange, linewidth=2, marker=:utriangle, markersize=3, markercolor=:orange, markerstrokecolor=:orange)
plot!(p_NMSE, coefficients, DVR, label="DVR", color=:purple, linewidth=2, marker=:utriangle, markersize=3, markercolor=:purple, markerstrokecolor=:purple)
plot!(p_NMSE, coefficients, GMP_MDDR, label="GMP_MDDR", color=:gray, linewidth=2, marker=:utriangle, markersize=3, markercolor=:gray, markerstrokecolor=:gray)
plot!(p_NMSE, coefficients, hybrid, label="Hybrid", color=:black, linewidth=2, marker=:utriangle, markersize=3, markercolor=:black, markerstrokecolor=:black)
xlabel!("Number of Coefficients")
ylabel!("Minimum NMSE (dB)")
title!("NMSE Comparison Among Models")
plot!(legend=:topright, right_margin=2.9Plots.cm, marker=:utriangle, markercolor=:white, markerstrokecolor=:blue)
plot!(xticks=0:20:100, yticks=-46:4:-30)
xlims!(0, 101)
ylims!(-46, -30)
display(p_NMSE)

In [ ]:
colors = Dict(
    "without DPD" => :brown,
    "GMP" => :red,
    "2nd_DDR" => :blue,
    "MDDR" => :green,
    "CRV" => :orange,
    "DVR" => :purple,
    "GMP_MDDR" => :gray,
    "Hybrid" => :black
)
names = ["without DPD", "GMP", "2nd_DDR", "MDDR", "CRV", "DVR", "GMP_MDDR", "Hybrid"]
Fs=300e6
NFFT=256
main_channel_band=[-28, 28]
adjacent_channel_bands=[[-39, -37], [38, 40]]
detrend=nothing
window=Windows.hanning
noverlap=0
psd = plot(legend=true)
for n = 1:size(yn, 2)
    if detrend != nothing
        yn[:, n] = DSP.detrend(yn[:, n])
    end
    w = window(NFFT)
    pgram = DSP.welch_pgram(yn[:, n], NFFT, noverlap; window=w, fs=Fs)
    Pxx = pgram.power ./ maximum(pgram.power)
    freqs = pgram.freq / 1e6
    main_power = mean(Pxx[(freqs .> main_channel_band[1]) .& (freqs .< main_channel_band[2])])
    for (i, band) in enumerate(adjacent_channel_bands)
        adj_power = mean(Pxx[(freqs .> band[1]) .& (freqs .< band[2])])
        acpr_value = 10 * log10(adj_power / main_power)
        println("ACPR $n (Band $i): $acpr_value dB")
    end
    plot!(psd, freqs, 10 * log10.(Pxx), label=names[n], color=colors[names[n]], linewidth=2, xlabel="Frequency (MHz)", ylabel="Normalized Power Spectral Density (dB)", grid=false, seriestype=:line)
end
display(psd)

## Write data

In [ ]:
# Write the GMP DPD model with caution because it will overwrite previously saved data
# data_w = matopen("w_GMP.mat", "w")
# write(data_w, "w_GMP", w)
# close(data_w)